Read the .res file

In [21]:
%load_ext autoreload
%autoreload 2
import json
from scipy import sparse
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import progress_bar
from collections import defaultdict
import string
from sys import getsizeof
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# %store -r dataset
# dataset.shape[0]

dataset = pd.read_json("signalmedia-1m.jsonl", lines=True)
dataset.shape[0]

1000000

In [ ]:
getsizeof(dataset)

In [ ]:
queryTitleDoc = defaultdict()
for i in progress_bar.iter_progress(range(dataset.shape[0])):
    docid = dataset["id"].iloc[i]
    queryTitleDoc[int(i)] = docid

In [ ]:
print(len(queryTitleDoc))

In [ ]:
with open('queryTitleDoc.json', 'w') as fp:
    json.dump(queryTitleDoc, fp)

In [ ]:
with open('queryTitleDoc.json') as json_data:
    qTitleD = json.load(json_data)
    print(len(qTitleD))
    print(qTitleD["2"])

### Query classes
1. We consider news headlines/title as queries, with a single relevant results, the corresponding article.
2. We perform ranking on the (whole) collection, and we can have the following scenarios:

    a) R@1 = 1, i.e., the top document retrieved is the correct one
    
    b) R@1 != 1 but R@10 = 1, i.e., the correct document is not the first, but among the first 10
    
    c) else

### Results from Terrier BM25 have this shape
    0 Q0 f7ca322d-c3e8-40d2-841f-9d7250ac72ca 0 67.80570840241509 BM25b0.75
    0 Q0 5ed3ed90-b025-4f90-98ab-48ac1182f3a7 1 33.68442635555825 BM25b0.75
    0 Q0 80cade5d-dfbb-4887-8700-34032ed708d9 2 33.14615405559586 BM25b0.75
    0 Q0 84947a2e-31ea-41a6-a63c-ba2b4bde2caa 3 31.083372155166458 BM25b0.75
    0 Q0 d1521e9f-ca2c-4ca0-9179-4966d508ebc2 4 30.724469746013728 BM25b0.75
    0 Q0 ddcda110-461f-4608-8509-b4080e80c305 5 30.02468908628416 BM25b0.75
    0 Q0 17e25f0c-7afe-480d-a270-2b477c0033e6 6 29.665149620555425 BM25b0.75
    0 Q0 9752f0cf-61ae-46ee-8494-6edb03b7d24d 7 29.66147722262594 BM25b0.75
    0 Q0 df87f8ee-9904-4f67-9590-5d0fa2cd2adb 8 29.460120040217348 BM25b0.75
    0 Q0 1738286d-9853-4b53-aab1-f418835ac1e3 9 29.444973786956144 BM25b0.75

In [ ]:
queryType = defaultdict(list) # the key is A, B or C and the values are list of queryId

In [ ]:
i = 0 

with open("BM25-signal.res", "r") as f:
    for line in progress_bar.iter_progress(f):
        i+=1
        if i % 10000 ==0 :
            print("Processed: ", i)
        resultLine = line.split()
        queryId = int(resultLine[0])
        if not int(resultLine[3]) > 100: 
            if int(resultLine[3]) == 0:  # class A
                if resultLine[2] == queryTitleDoc[queryId]:
                    queryType["A"].append(queryId)
                    continue
            elif int(resultLine[3]) < 10: # class B
                if resultLine[2] == queryTitleDoc[queryId]:
                    queryType["B"].append(queryId)
                    continue
            elif int(resultLine[3]) < 100: # class C
                 if resultLine[2] == queryTitleDoc[queryId]:
                    queryType["C"].append(queryId)
                    continue
        else:
            if resultLine[2] == queryTitleDoc[queryId]:                
                queryType["D"].append(queryId)
#                 print(line)

            

In [ ]:
for qType, qList in queryType.items():
    print(qType, len(qList), len(set(qList)))
    
print(i)
print(line)

In [ ]:
all = set(range(1459))

In [ ]:
len(all - set(queryType["A"]) - set(queryType["B"]) - set(queryType["C"]) - set(queryType["D"]))
# these are queries which don't have the CORRECT Doc retrieved at all... e.g. query 206

In [ ]:
queryTitleDoc[206] 

In [ ]:
dataset.iloc[206]

In [ ]:
dataset["content"].iloc[206]

In [ ]:
dataset.iloc[206]["id"]

## Query types on Terrier

* ('A', 390058)
* ('C', 131121)
* ('B', 302560)
* ('D', 64843)
* No docs in top 1000 results: 111418

Put of a total of 1.000.000 we have the 4 types of queries summing up to 888582. 

All the other queries don't have the CORRECT doc in the retrieved results. See example above: article 206


```
            if not int(resultLine[3]) > 1000:
                if int(resultLine[3]) == 0:  # class A
                    if resultLine[2] == queryTitleDoc[queryId]:
                        queryType["A"].append(queryId)
                        continue
                elif int(resultLine[3]) < 10: # class B
                    if resultLine[2] == queryTitleDoc[queryId]:
                        queryType["B"].append(queryId)
                        continue
                elif int(resultLine[3]) < 100: # class C
                    if resultLine[2] == queryTitleDoc[queryId]:
                        queryType["C"].append(queryId)
                        continue
                elif int(resultLine[3]) < 1000: # class C
                    if resultLine[2] == queryTitleDoc[queryId]:
                        queryType["D"].append(queryId)
            else:
                if resultLine[2] == queryTitleDoc[queryId]:
                    queryType["E"].append(queryId)
                    ```

In [ ]:
with open('queryTypeTerrier.json') as json_data:
    qTypeTerrier = json.load(json_data)
    print(len(qTypeTerrier))
    print(len(qTypeTerrier["A"]))

## Query types on Elastic

They come from 5 files each with 200.000 queries (we split the 1 mil for speedier processing)

0-200.000
('A', 83481)
('C', 24437)
('B', 58630)
('D', 11651)

200-400
('A', 82510)
('C', 24669)
('B', 59408)
('D', 11743)

400-600
('A', 82569)
('C', 24853)
('B', 59332)
('D', 11668)

600-800
('A', 81463)
('C', 25049)
('B', 59921)
('D', 11734)

800-1000
('A', 81103)
('C', 25225)
('B', 60310)
('D', 11628)

In [ ]:
with open('queryTypeElastic200.json') as json_data:
    qTypeElastic200 = json.load(json_data)
    print(len(qTypeElastic200))
    print(len(qTypeElastic200["A"]))
    
with open('queryTypeElastic400.json') as json_data:
    qTypeElastic400 = json.load(json_data)
    print(len(qTypeElastic400))
    print(len(qTypeElastic400["A"]))
    
with open('queryTypeElastic600.json') as json_data:
    qTypeElastic600 = json.load(json_data)
    print(len(qTypeElastic600))
    print(len(qTypeElastic600["A"]))

with open('queryTypeElastic800.json') as json_data:
    qTypeElastic800 = json.load(json_data)
    print(len(qTypeElastic800))
    print(len(qTypeElastic800["A"]))
    
with open('queryTypeElastic1000.json') as json_data:
    qTypeElastic1000 = json.load(json_data)
    print(len(qTypeElastic1000))
    print(len(qTypeElastic1000["A"]))
    

In [ ]:
# let's try some intersections and make sure they are null
print(len(set(qTypeElastic200["A"]).intersection(set(qTypeElastic1000["D"]))))

# we shall unite!
print(len(set(qTypeElastic200["A"]))+len(set(qTypeElastic400["A"])))
print(len(set(qTypeElastic200["A"]).union(set(qTypeElastic400["A"]))))

qTypeElastic = defaultdict()
qTypeElastic["A"] = set(qTypeElastic200["A"]) | set(qTypeElastic400["A"]) | set(qTypeElastic600["A"]) | set(qTypeElastic800["A"]) | set(qTypeElastic1000["A"])
qTypeElastic["B"] = set(qTypeElastic200["B"]) | set(qTypeElastic400["B"]) | set(qTypeElastic600["B"]) | set(qTypeElastic800["B"]) | set(qTypeElastic1000["B"])
qTypeElastic["C"] = set(qTypeElastic200["C"]) | set(qTypeElastic400["C"]) | set(qTypeElastic600["C"]) | set(qTypeElastic800["C"]) | set(qTypeElastic1000["C"])
qTypeElastic["D"] = set(qTypeElastic200["D"]) | set(qTypeElastic400["D"]) | set(qTypeElastic600["D"]) | set(qTypeElastic800["D"]) | set(qTypeElastic1000["D"])

In [ ]:
print(type(qTypeElastic))
for key, itemlist in qTypeElastic.items():
    qTypeElastic[key] = sorted(list(itemlist))
    print(key, len(itemlist))
# the numbers are ok

# dumping to file
with open('queryTypeElastic.json', 'w') as fp:
    json.dump(qTypeElastic, fp)
    

In [ ]:
print(type(qTypeElastic["A"]))
print(qTypeElastic["A"][:10])
print(qTypeTerrier["A"][:10])
print(len(qTypeElastic))

## We intersect the 2 SE

- we intersect A,B,C,D and decide upon the final queries
- we run there queries on BM25P10 - idf and tfidf and count:
    - how many intersect in A
    - how many moved from B to A
    - how many move from C to A and B and so on


In [ ]:
# let's do some intersections
queryTypeGlobal = defaultdict()
for keyT, itemlistT in qTypeTerrier.items():
    itemlistE = qTypeElastic[keyT]
    itemlistG = sorted(list(set(itemlistT) & set(itemlistE)))
    print(keyT)
    print("Terrier: ", len(itemlistT), " Elastic: ", len(itemlistE), " Global: ",len(itemlistG))
    queryTypeGlobal[keyT] = itemlistG

In [ ]:
# save global
with open('queryTypeGlobal.json', 'w') as fp:
    json.dump(queryTypeGlobal, fp)

# Sample queries and qid:title

In [45]:
with open('queryTypeGlobal.json') as json_data:
    qTypeGlobal = json.load(json_data)

In [47]:
for k,v in qTypeGlobal.items():
    print(k, len(v))

A 331954
C 71573
B 206440
D 30420


In [46]:
# Create random samples of queries and write them to file!

for qType in ["A","B","C","D"] :  
    sample_10000 = random.sample(qTypeGlobal[qType], 15000)
    with open("sample-15000-"+ qType + ".txt", 'w') as fp:
        json.dump(sample_10000, fp) # we are writing a list

In [17]:
with open('sample-15000-A.txt') as json_data:
    sample10000A = json.load(json_data)
    print(len(sample10000A))
    print(sample10000A[:20])

15000
[628127, 975021, 897138, 737936, 605110, 765994, 162772, 792097, 161966, 294054, 433510, 997633, 928796, 556438, 242711, 686115, 690237, 385208, 280346, 392234]


In [ ]:
newTitle = dataset.title.str.replace('[^a-zA-Z ]', '')
newTitle[0:100]

In [ ]:
def gen_sample_queries(newTitle, qtype):
    sampleFilename = "sample-15000-"+ qtype + ".txt"
    with open(sampleFilename) as json_data:
        sample_10000 = json.load(json_data)
    with open("signal-title-queries-sample-15000-"+ qtype + ".txt", "w") as f:
        for qid in sample_10000:
            f.write(str(qid)+":"+newTitle[qid]+"\n")

In [ ]:
for qt in ["A", "B", "C", "D"]:
    gen_sample_queries(newTitle, qt)

# 6. Generate query groups according to HARDNESS

In [31]:
%store -r signal_query_hardness_dict

In [40]:
query_hardness_list = list(signal_query_hardness_dict.items())
sorted_query_hardness_list = sorted(query_hardness_list, key=lambda x: x[1])

a,b,c = np.array_split(np.array(sorted_query_hardness_list),3)

print(len(a),len(b), len(c))
np.set_printoptions(suppress=True)
print(a[-1], b[-1], c[-1])
print(a[:,0])

333334 333333 333333
[ 61653.              0.02506266] [ 697878.               0.05928854] [ 9237.    22.]
[   234.    792.   1220. ...,  33639.  50879.  61653.]


In [41]:
# We look at the intervals from above
h = a[:,0].astype(int).tolist()
m = b[:,0].astype(int).tolist()
e = c[:,0].astype(int).tolist()

print(len(h),len(m), len(e))

333334 333333 333333


In [42]:
for qType in ["A","B","C","D"] :
    print(qType)
    group_intersect_H = set(qTypeGlobal[qType]) & set(h)
    group_intersect_M = set(qTypeGlobal[qType]) & set(m)
    group_intersect_E = set(qTypeGlobal[qType]) & set(e)
    
    print(len(group_intersect_H))
    print(len(group_intersect_M))
    print(len(group_intersect_E))
    
    sample_H1000 = random.sample(group_intersect_H, 1000)
    sample_M1000 = random.sample(group_intersect_M, 1000)
    sample_E1000 = random.sample(group_intersect_E, 1000)

    with open("sample-H1000-"+ qType + ".txt", 'w') as fp:
        json.dump(sample_H1000, fp) # we are writing a list
    with open("sample-M1000-"+ qType + ".txt", 'w') as fp:
        json.dump(sample_M1000, fp) # we are writing a list
    with open("sample-E1000-"+ qType + ".txt", 'w') as fp:
        json.dump(sample_E1000, fp) # we are writing a list

A
91172
122949
117833
B
69101
81478
55861
C
27708
22935
20930
D
13239
6449
10732


In [43]:
def gen_sample_queries(newTitle, qtype, hardness):
    sampleFilename = "sample-"+hardness+"1000-"+ qtype + ".txt"
    with open(sampleFilename) as json_data:
        sample_10000 = json.load(json_data)
    with open("signal-title-queries-sample-"+hardness+"1000-"+ qtype + ".txt", "w") as f:
        for qid in sample_10000:
            f.write(str(qid)+":"+newTitle[qid]+"\n")

In [44]:
for qType in ["A","B","C","D"] :
    for hardness in ["E", "M", "H"]:
        gen_sample_queries(newTitle, qType, hardness)